# Pandas - Reading data

This notebook is the second part of the collection devoted to the pandas library.

It explores the ways how data can be imported into DataFrames. 

More details can be found in the official documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/io.html#io-sql

Most of the functions for reading data are named `pandas.read_XXX`, where XXX is the format used. We will go through the most commonly used ones.

### This tutorial data sets origins

* IMDB: https://datasets.imdbws.com/
* Box office (Kaggle / TMDB): https://www.kaggle.com/c/tmdb-box-office-prediction/data
* Awards: https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films
* Rotten Tomatoes: https://data.world/prasert/rotten-tomatoes-top-movies-by-genre
* Guardian: https://www.theguardian.com/news/datablog/2010/oct/16/greatest-films-of-all-time
* Wikipedia movies: https://github.com/prust/wikipedia-movie-data


In [1]:
# Necesssary import evil

import jupy_helpers
import pandas as pd
from IPython.display import display

In [2]:
# List function for input in pandas.

print("\n".join(method for method in dir(pd) if method.startswith("read_")))

read_clipboard
read_csv
read_excel
read_feather
read_fwf
read_gbq
read_hdf
read_html
read_json
read_msgpack
read_parquet
read_pickle
read_sas
read_sql
read_sql_query
read_sql_table
read_stata
read_table


## Read CSV

Nowadays, a lot of data comes in the textual Comma-separated values format (CSV).
Although not properly standardized, it is the de-facto standard for files that are not
huge and are meant to be read by human eyes too.

Let's read the ratings of several (hundred) movies from Rotten Tomatoes:

In [3]:
%head ../data/rotten_tomatoes_top_movies_2019-01-15.csv 10

Rank,Title,RatingTomatometer,No. of Reviews,Genres
1,Black Panther (2018),97,444,action|adventure
2,Mad Max: Fury Road (2015),97,394,action|adventure
3,Wonder Woman (2017),93,410,action|adventure
4,Metropolis (1927),99,118,action|adventure
5,Coco (2017),97,308,action|adventure
6,Dunkirk (2017),92,413,action|adventure
7,Incredibles 2 (2018),94,332,action|adventure
8,Star Wars: The Last Jedi (2017),91,419,action|adventure
9,Logan (2017),93,371,action|adventure



In [4]:
rotten_df = pd.read_csv("../data/rotten_tomatoes_top_movies_2019-01-15.csv")
rotten_df.head(9)

,Rank,Title,RatingTomatometer,No. of Reviews,Genres
0,1,Black Panther (2018),97,444,action|adventure
1,2,Mad Max: Fury Road (2015),97,394,action|adventure
2,3,Wonder Woman (2017),93,410,action|adventure
3,4,Metropolis (1927),99,118,action|adventure
4,5,Coco (2017),97,308,action|adventure
5,6,Dunkirk (2017),92,413,action|adventure
6,7,Incredibles 2 (2018),94,332,action|adventure
7,8,Star Wars: The Last Jedi (2017),91,419,action|adventure
8,9,Logan (2017),93,371,action|adventure


The automatic data type parsing automatically converts columns to appropriate types:

In [5]:
rotten_df.dtypes

Rank                  int64
Title                object
RatingTomatometer     int64
No. of Reviews        int64
Genres               object
dtype: object

Sometimes the CSV input does not work out of the box. Although pandas automatically understands and reads zipped files,
it usually does not automatically infer the file format - for details, see the `read_csv` documentation here: 
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

In [6]:
pd.read_csv('../data/title.basics.tsv.gz')

ParserError: Error tokenizing data. C error: Expected 2 fields in line 4, saw 3


In [26]:
!zcat../data/title.basics.tsv.gz | head -n 10

/usr/bin/sh: 1: zcat../data/title.basics.tsv.gz: not found


...in this case, the CSV file does not use commas to separate values. Therefore, we need to specify a few more arguments:

In [11]:
imdb_titles = pd.read_csv('../data/title.basics.tsv.gz', sep='\t')
imdb_titles.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


Noticed the `\N` endYear values?

**Exercise:** Use `na_values` argument to mark `\N` as a null (missing) value. 

In [14]:
%exercise

imdb_titles = pd.read_csv('../data/title.basics.tsv.gz', sep='\t', na_values=[r"\N"])

In [15]:
%validate

assert pd.isna(imdb_titles.loc[0, 'endYear'])

In [17]:
imdb_titles.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894.0,NaN,1.0,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892.0,NaN,5.0,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892.0,NaN,4.0,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892.0,NaN,NaN,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,NaN,1.0,"Comedy,Short"


See the difference?

In [18]:
imdb_titles.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894.0,NaN,1.0,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892.0,NaN,5.0,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892.0,NaN,4.0,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892.0,NaN,NaN,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,NaN,1.0,"Comedy,Short"


## Read Excel

Let's read the list of best movies by genre from Guardian (a bit old, written in 2010).

![Screenshot](guardian-best-horrors.png)

In [19]:
pd.read_excel("../data/guardian-greatest_films_of_all_time.xlsx")

,Entry,Film,Director,Leading actors,Year of cinema release,No of Oscars won,IMDB link,Guardian film page,Country
0,1,Brief Encounter,David Lean,"Celia Johnson, Cyril Raymond, Stanley Holloway...",1945,NaN,http://www.imdb.com/title/tt0037558/,http://www.guardian.co.uk/film/movie/35664/bri...,UK
1,2,Casablanca,Michael Curtiz,"Claude Rains, Humphrey Bogart, Ingrid Bergman,...",1942,3.0,http://www.imdb.com/title/tt0034583/,http://www.guardian.co.uk/film/movie/36156/cas...,USA
2,3,Before Sunrise,Richard Linklater,Ethan Hawke and Julie Delpy,1995,NaN,http://www.imdb.com/title/tt0112471/,http://www.guardian.co.uk/film/movie/58808/bef...,USA
3,3,Before Sunset,Richard Linklater,Ethan Hawke and Julie Delpy,2004,NaN,http://www.imdb.com/title/tt0381681/awards,http://www.guardian.co.uk/film/movie/101181/be...,USA
4,4,Breathless,Jean-Luc Godard,"Jean Seberg, Jean-Paul Belmondo",1960,NaN,http://www.imdb.com/title/tt0053472/,http://www.guardian.co.uk/film/movie/36219/a-b...,France
5,5,In the Mood for Love,Kar Wai Wong,"Maggie Cheung Man-Yuk, Rebecca Pan, Tony Leung...",2000,NaN,http://www.imdb.com/title/tt0118694/,http://www.guardian.co.uk/film/movie/85442/in....,Hong Kong
6,6,The Apartment,Billy Wilder,"Fred MacMurray, Jack Lemmon, Ray Walston, Shir...",1960,NaN,http://www.imdb.com/title/tt0053604/,http://www.guardian.co.uk/film/movie/36225/apa...,USA
7,7,Hannah & Her Sisters,Woody Allen,"Barbara Hershey, Carrie Fisher, Dianne Wiest, ...",1986,3.0,http://www.imdb.com/title/tt0091167/,http://www.guardian.co.uk/film/movie/89162/han...,USA
8,8,Eternal Sunshine of the Spotless Mind,Michel Gondry,"Elijah Wood, Jim Carrey, Kate Winslet, Kirsten...",2004,1.0,http://www.imdb.com/title/tt0338013/,http://www.guardian.co.uk/film/movie/100140/et...,USA
9,9,Room With a View,James Ivory,"Helena Bonham Carter, Julian Sands, Maggie Smith",1985,3.0,http://www.imdb.com/title/tt0091867/,http://www.guardian.co.uk/film/movie/77615/roo...,UK


Hmmmmph... Pandas parsed just the first spreadsheet. Let's see what are the options. If in doubt, look in the documentation:
https://pandas.pydata.org/pandas-docs/stable/reference/io.html#excel

In [21]:
xlsx = pd.io.excel.ExcelFile("../data/guardian-greatest_films_of_all_time.xlsx")
xlsx

In [22]:
xlsx.sheet_names

['ROMANCE',
 'CRIME',
 'COMEDY',
 'ACTION',
 'ARTHOUSE AND DRAMA',
 'SCI-FI AND FANTASY',
 'HORROR']

In [ ]:
xlsx.parse("HORROR")

In [ ]:
%exercise

crimes =...                    # Find the table of crime movies
tenth_best = crimes.loc[...]   # Find the 10-th best crime movie
movie_name = ...               # Get the name of the movie

# display
movie_name

In [ ]:
%validate

assert movie_name[7:9] == "la"

## Read JSON

In [ ]:
wiki_movies = pd.read_json("../data/wikipedia-movies.json")
wiki_movies.head(10)

## Read SQL

On its own, pandas can read SQLite databases. If **sqlalchemy** package is installed, pandas allows to access
any database that is supported by the former library.

In [ ]:
# This requires sqlalchemy
award_table = pd.read_sql("awards", con='sqlite:///../data/awards.sqlite')
award_table.tail(20)

In [ ]:
# It is possible to pass a SQL query too (no sqlalchemy necessary with sqlite3)
import sqlite3
connection = sqlite3.connect("../data/awards.sqlite")

awards2017 = pd.read_sql("SELECT * FROM awards WHERE Year=2017", con=connection)
awards2017

## Read HTML

Pandas is able to scrape data from tables embedded in web pages using the `read_html` function.
This might or might not bring you good results and probably you will have to tweak your
data frame manually. But it is a good starting point - much better than being forced to parse
the HTML ourselves!

Let's download a list of highest-grossing films from wikipedia!

In [ ]:
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_highest-grossing_films")
type(tables), len(tables)

Does the page really contain 95 tables? The number is quite high and we must check which of the tables
are meaningful and which are not. We are mostly interested in the first displayed one.

**Exercise:** Find **i** to obtain the right table:

In [ ]:
%exercise

i = ...

table = tables[i]
table.head(10)

In [ ]:
%validate

assert table.iloc[2]["Title"] == "Titanic"  # 3rd msot grossing movie ever

## Write CSV

Pandas is able to write to many various formats but the usage is similar. 

In [ ]:
award_table.to_csv("awards.csv", index=False)

In [ ]:
%head awards.csv 10

## Write SQL

Load all data for the rest of the workshop and save as into local sqlite database.

**Note**: This is an important step. We will use the data in the later phases.
If in doubt, refer to the "solution" version of this file (TODO: link).

In [23]:
workshop_data = dict(
    imdb_titles = pd.read_csv('../data/title.basics.tsv.gz', sep='\t', na_values="\\N"),
    imdb_ratings = pd.read_csv('../data/title.ratings.tsv.gz', sep='\t', na_values="\\N"),
    boxoffice = pd.read_csv('../data/boxoffice_march_2019.csv.gz'),
    rotten_tomatoes = pd.read_csv("../data/rotten_tomatoes_top_movies_2019-01-15.csv"),
    awards = pd.read_sql("awards", con='sqlite:///../data/awards.sqlite')
)

In [24]:
con = 'sqlite:///./workshop_data.sqlite'

for name, df in workshop_data.items():
    df.to_sql(name, con, if_exists="replace", index=False)

Note: When done with this notebook, we suggest that you shutdown the kernel to free the memory.